# TO DO 
- Formulate a general format for the data set 
- Update the preprocessing properly 
- A trip is a class 
- A schedule is also a class which hold the instances of trip 


Schedule (df, start_leg, no_of_trips, etc..)

Trip (steering_duty, spread_over, ...)

### SET UP AND IMPORTS 

In [12]:
# SET UP AND IMPORTS 
import pandas as pd 
import datetime
import time

df = pd.read_excel('dataset/processed/psl.xlsx').reset_index(drop=True)
 
# Adding the Terminal Gap and Duty only for processing 
df['Terminal Gap'] = [0] * len(df)
df['Duty'] = [0] * len(df)

START_PLACE = 'PSL'
#start_legs contain the legs with departure place as start_place
start_legs = df[df['Departure Place'] == START_PLACE].sort_values('Departure Time', ascending=True)
start_legs.reset_index(drop=True,inplace=True)
#main_legs contains all other legs 
# main_legs = df[df['Departure Place'] != START_PLACE]
# MIN_DUTY = 27000 #setting the minimum duty as 7:00 hrs 
MIN_DUTY = 27000 #setting the minimum duty as 7:00 hrs 
# The ideal duty is 7:30 hrs with a 15 min break at the start and end . Then total 8:00 
# MAX_DUTY = 30600 # setting the maximum duty as 8:30 hrs
MAX_DUTY = 36000 # setting the maximum duty as 10:00 hrs
MIN_TERMINAL_GAP = 300
MAX_TERMINAL_GAP = 900

BREAK = 1800 # break time of 30 min to avoid 5hrs continuous working
IS_BREAK = False # setting the break status as false by default 

MAX_SPLIT = 3600 # Setting the maximum split time as 60 min 
MIN_TEMPSET_SIZE = 2 # define the minimum number of legs in a temp_set


# FOR THE PURPOSE OF MEASURING THE PERFORMANCE 
success = 0 # Not necessary , just to count the number of start legs taken 
trips_count = 0 # count the number of trips 
exe_generateTempSet  = 0
exe_popTempSet  = 0


### SUPPORTING FUNCTIONS 

#### Generate Tempset 
- Below function creates the temset for a particular leg in the stack . It checks for the needed criteria and then creates the set of legs that are possibly gives us a feasible solution 

In [13]:
# SUPPORTING FUNCTIONS
def generateTempSet(leg) -> pd.DataFrame:
    # FN LOGIC IS NOT CORRECT , NEED TO CREATE THE TEMP SET ONLY WITH THE CRITERIA
    # SORTING THEM AFTER CREATING THE ENTIRE TEMPSET IS NOT OPTIMAL
    global exe_generateTempSet
    x = time.time()

    # Creating a copy of the main legs.Can be limited with steering duty remaining 
    temp_set_df = df[(df['Departure Place'] == leg['Arrival Place'])
                  & (df['Departure Time'] > leg['Arrival Time act'])]
    temp_set_df = temp_set_df.sort_values(
        'Departure Time', ascending=True).reset_index(drop=True)
    temp_set_df['Terminal Gap'] = temp_set_df['Departure Time'] - leg['Arrival Time act']
    # Sorting and resetting index
    # '''BELOW LINE MAY BE NOT NECESSARY'''
    # display(temp_set)
    temp_set = pd.DataFrame()
    # temp_set = temp_set_df[
    #     (temp_set_df['Terminal Gap'] >= MIN_TERMINAL_GAP) & (
    #         temp_set_df['Terminal Gap'] < MAX_TERMINAL_GAP)  # |
    #     # (temp_set['Duty'] >= MIN_DUTY) & (temp_set['Duty'] <= MAX_DUTY)
    # ]
# THERE ARE CHANCES THAT THE TEMP SET MAY BE EMPTY , SO WE ARE TRYING FOR 
#  THE NEXT LEVEL SEARCHING
    terminal_gap = MAX_TERMINAL_GAP
    ADD_ON = 900 # 15 min
    # NEED TO OPTIMIZE BELOW LOOP AS EVERY TIME IT LOOPS THROUGH THE ENTIRE LEGS TILL
    # THE SPECIFIED TERMINAL GAP , CAN USE EXTEND OR CONCAT TO ITERATIVELY ADDING UP THE 
    # TEMP SET  
    while len(temp_set) < MIN_TEMPSET_SIZE and terminal_gap < MAX_SPLIT: # 1 hr split
        temp_set = temp_set_df[
            (temp_set_df['Terminal Gap'] >= MIN_TERMINAL_GAP) & (
                temp_set_df['Terminal Gap'] < terminal_gap)  # |
            # (temp_set['Duty'] >= MIN_DUTY) & (temp_set['Duty'] <= MAX_DUTY)
        ]
        terminal_gap += ADD_ON
    # temp_set['Duty'] = leg['Duty'] + temp_set['Running Time'] + temp_set['Terminal Gap']
    temp_set.loc[:, 'Duty'] = leg['Duty'] + temp_set['Running Time'] + temp_set['Terminal Gap'] # check syntax 
    
    y = time.time()
    exe_generateTempSet += (y - x)
    # if len(temp_set) < 2 : print(f"Size of tempset == {len(temp_set)}")
    return temp_set

#### Pop Tempset 
- This function pops the currently visited leg from the corresponding tempset, then the tempset will always contain the possible legs from a given leg that satisfies the criteria needed   

In [14]:
def popTempSet(temp_set) -> pd.DataFrame:
    global exe_popTempSet
    x = time.time()

    temp_set.drop(0, inplace=True)
    # print("popTempSet")
    # display(temp_set)
    temp_set.reset_index(inplace=True)
    temp_set.drop('index',axis= 1, inplace=True)

    y = time.time()
    exe_popTempSet += (y - x)
    return temp_set


#### Display Trip
- Returns the trip once the stack is satisfying all the criteria 

In [15]:
def displayTrip(stack):
    global trips_count
    frame = pd.DataFrame()
    for x in stack:
        frame = pd.concat([frame, (x['current_leg'].to_frame()).T], ignore_index=True)
    frame['Terminal Gap'] = frame['Terminal Gap'].shift(-1)
    duty = second=frame['Duty'].iloc[len(frame) - 1]
    hrs, rem = divmod(duty, 3600)
    mint, sec = divmod(rem , 60)
    duty = datetime.time(hour=hrs, minute=mint, second= sec)
    frame = frame.drop('Duty', axis=1)

    trips_count += 1
    return [frame, duty]

def seconds_to_time(df, columns):
    df[columns] = df[columns].apply(lambda x: pd.to_datetime(x, unit='s').dt.strftime('%H:%M:%S'))


In [16]:
# check_df = pd.DataFrame(
#     {'Sl No.': [1, 124, 313, 641, 336, 695],
#      'Departure Time': [13200, 13800, 18300, 22500, 29400, 36300],
#      'Departure Place': ['PSL', 'KLKV', 'TVM', 'KNVLA', 'MC', 'KLKV'],
#      'Route of Operation': ['NH',
#                             'NH',
#                             'NH-UDA',
#                             'UDA',
#                             'NH',
#                             'PVR-VZM-BYPASS-TVM-NH-KLKV'],
#      'Arrival Place': ['KLKV', 'TVM', 'KNVLA', 'MC', 'KLKV', 'PSL'],
#      'Arrival Time act': [13500, 17700, 21900, 28800, 36000, 48900],
#      'KM': [3.5, 33.7, 32.0, 39.0, 40.0, 81.5],
#      'Running Time': [300, 3900, 3600, 6300, 6600, 12600],
#      'Terminal Gap': [300, 600, 600, 600, 300, None]}
# )



#### Remove Legs
- Once a leg is taken for a trip , it should be removed so that it will not be considered for the next round of iterations  

In [17]:
def removeLegs(trip) -> None:
    global success
    for i in range(len(trip)):
        if trip.iloc[i]['Departure Place'] == START_PLACE:
            # remove from both df and start_legs 
            df.drop((df[df['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            start_legs.drop((start_legs[start_legs['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            df.reset_index(drop=True, inplace=True)
            start_legs.reset_index(drop=True, inplace=True)
            success += 1
        else:
            # remove only from df 
            df.drop((df[df['Sl No.'] == trip.iloc[i]['Sl No.']]).index, axis= 0, inplace=True)
            df.reset_index(drop=True, inplace=True)
        



#### Backtracking 

In [18]:
def backtrack(stack):
    #  If temp_set is empty , it means that we cannot go to any other places from the 
    # last leg , so we can pop the last leg in the stack  
    stack.pop()
    # After popping stack top , there are also chances that the temp_set of top elements 
    # in the stack may be empty, so those ones also should be popped out, because we cannot 
    # goto anywhere else from there  
    while(stack[-1]['temp_set'].empty) :
            stack.pop()
    stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
    stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])


### ALGORITHM

In [19]:
# ALGORITHM
def generateTrip(start_leg):
    global error
    # Making the break flag False initially 
    IS_BREAK = False
    try:
        # Initializing stack with start leg . stack contain the legs in the trip 
        stack = [{"current_leg": start_leg, "temp_set": pd.DataFrame()}] 
        # Generating the temp_set for the start_leg . temp_set contains the legs that can be 
        # visited from the start leg
        # BELOW 2 LINES CAN BE PUT INSIDE THE LOOP  
        temp_set = generateTempSet(start_leg)
        stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})

        while len(stack) > 1 : 
            temp_set = generateTempSet(stack[-1]['current_leg'])
            if temp_set.empty :
                # EXCEPTION CASES CAN BE HANDLED HERE , MEANS WE CAN CHECK WHETHER THE 
                # STACK GETS EMPTY ON BACKTRACKING TO AVOID OUT OF BOUND 
                backtrack(stack)
                # After backtracking a new leg is replaced on the top , so we can just continue
                continue
            stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})


            if (top_leg:=stack[-1]['current_leg'])['Arrival Place'] == START_PLACE:
                if MIN_DUTY > top_leg['Duty']:
                    # NEED TO ADDRESS SHUTTLE TRIP ISSUE OF START_PLACE
                    while(stack[-1]['temp_set'].empty) :
                        stack.pop()
                    stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
                    stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

                elif top_leg['Duty'] > MAX_DUTY:
                    stack.pop()
                    while(stack[-1]['temp_set'].empty) :
                        stack.pop()
                    stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
                    stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

                else: # MIN_DUTY <= top_leg['Duty'] <= MAX_DUTY 
                    #ideal case 
                    break;
        result = displayTrip(stack)
        removeLegs(result[0])
        display(result[0])

        # display(result[0].to_dict(orient='list')) # to display a df as a dict of list  
        print("Steering Duty = ", result[1]) 
           
    except Exception as e:
            # removeLegs(start_leg)
            start_legs.drop(0, axis=0, inplace=True)
            start_legs.reset_index(drop=True, inplace=True)
            print(f"Error occured = {e}")
            return 

In [20]:
# # ALGORITHM
# def generateTrip(start_leg):
#         stack = [{"current_leg": start_leg, "temp_set": pd.DataFrame()}] # contain the legs in the trip 
        
#         temp_set = generateTempSet(start_leg)
#         print(temp_set)
#         stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})

#         while len(stack) > 1 : # and stack[-1]['current_leg']['Arrival Place'] != START_PLACE
#             temp_set = generateTempSet(stack[-1]['current_leg'])
            
#             if temp_set.empty :
#                 stack.pop()
#                 while(stack[-1]['temp_set'].empty) :
#                         stack.pop()
#                 stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
#                 stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

#                 # display(displayTrip(stack))
#                 continue
#             stack.append({"current_leg": temp_set.iloc[0], "temp_set": popTempSet(temp_set)})


#             if (top_leg:=stack[-1]['current_leg'])['Arrival Place'] == START_PLACE:
#                 if MIN_DUTY > top_leg['Duty']:
#                     # NEED TO ADDRESS SHUTTLE TRIP ISSUE OF START_PLACE
#                     while(stack[-1]['temp_set'].empty) :
#                         stack.pop()
#                     stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
#                     stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

#                 elif top_leg['Duty'] > MAX_DUTY:
#                     stack.pop()
#                     while(stack[-1]['temp_set'].empty) :
#                         stack.pop()
#                     stack[-1]['current_leg'] = stack[-1]['temp_set'].iloc[0]
#                     stack[-1]['temp_set'] = popTempSet(stack[-1]['temp_set'])

#                 else: # MIN_DUTY <= top_leg['Duty'] <= MAX_DUTY 
#                     #ideal case 
#                     break;
#         result = displayTrip(stack)
#         removeLegs(result[0])
#         display(result[0])
#         print("Steering Duty = ", result[1]) 
           

### MAIN PART

In [21]:
# MAIN
x = time.time()

if __name__ == "__main__":
    # generateTrip(start_legs.iloc[0])
    # generateTrip(start_legs.iloc[1])
    # generateTrip(start_legs.iloc[2])
    while(not start_legs.empty):
        generateTrip(start_legs.iloc[0])
    print(f"Success legs = {success} ..... Missing {104 - success} routes  ")
    print(f"NUMBER OF TRIPS = {trips_count}")

y = time.time()
print(f"Time = {y - x}")
print(f"Time taken by tempset = {exe_generateTempSet}")
print(f"Time taken by popTempSet = {exe_popTempSet}")

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,13200,PSL,NH,KLKV,13500,3.5,300,300
1,124,13800,KLKV,NH,TVM,17700,33.7,3900,600
2,313,18300,TVM,NH-UDA,KNVLA,21900,32.0,3600,600
3,641,22500,KNVLA,UDA,MC,28800,39.0,6300,600
4,336,29400,MC,NH,KLKV,36000,40.0,6600,300
5,695,36300,KLKV,PVR-VZM-BYPASS-TVM-NH-KLKV,PSL,48900,81.5,12600,None


Steering Duty =  09:50:00


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,15,15600,PSL,NH,KLKV,15900,3.5,300,300
1,650,16200,KLKV,PVR-VZM-BYPASS,TVM,23400,45.0,7200,900
2,648,24300,TVM,NTA-MJ,KRKM,30300,34.0,6000,600
3,646,30900,KRKM,MJ-NTA,MC,36900,41.0,6000,900
4,380,37800,MC,NH,KLKV,43800,40.0,6000,600
5,727,44400,KLKV,NH,PSL,45000,3.5,600,900
6,5,45900,PSL,NH,KLKV,46500,3.5,600,300
7,70,46800,KLKV,NH,PSL,47400,3.5,600,300
8,54,47700,PSL,NH,KLKV,48300,3.5,600,600
9,88,48900,KLKV,NH,PSL,49500,3.5,600,600


Steering Duty =  09:55:00


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,4,16200,PSL,NH,KLKV,16800,3.5,600,600
1,129,17400,KLKV,NH,TVM,21300,33.7,3900,600
2,317,21900,TVM,NH,KLKV,26400,33.7,4500,600
3,163,27000,KLKV,NH,TVM,31800,33.7,4800,600
4,355,32400,TVM,NH,NTA,35400,20.7,3000,600
5,208,36000,NTA,NH,TVM,39000,20.7,3000,1200
6,395,40200,TVM,NH,KLKV,45600,33.7,5400,300
7,68,45900,KLKV,NH,PSL,46500,3.5,600,None


...

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,13,19800,PSL,NH,KLKV,20400,3.5,600,1200
1,140,21600,KLKV,NH,MC,27600,40.0,6000,600
2,657,28200,MC,KNMLA-VZM-PVR,KLKV,36600,55.0,8400,600
3,194,37200,KLKV,NH,TVM,42000,33.7,4800,600
4,371,42600,TVM,NH,KLKV,47400,33.7,4800,300
5,74,47700,KLKV,NH,PSL,48000,3.5,300,None


Steering Duty =  07:40:00
Error occured = list index out of range


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,679,20400,PSL,KLKV-PVR-VZM-BYPASS,TVM,28200,48.5,7800,600
1,680,28800,TVM,NH,KLKV,33900,33.7,5100,600
2,202,34500,KLKV,NH-TVM,MC,40500,40.0,6000,900
3,399,41400,MC,NH,KLKV,47400,40.0,6000,300
4,239,47700,KLKV,NH,TVM,52500,33.7,4800,600
5,427,53100,TVM,NH,KLKV,57900,33.7,4800,300
6,636,58200,KLKV,PZKNU,VLKA,60000,13.0,1800,600
7,640,60600,VLKA,PZKNU,PSL,62100,11.0,1500,None


Steering Duty =  09:25:00


KeyboardInterrupt: 

In [ ]:
df

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap,Duty
0,2,28500,PSL,NH,KLKV,29100,3.5,600,0,0
1,3,24000,PSL,NH,KLKV,24600,3.5,600,0,0
2,9,19800,PSL,NH,KLKV,20400,3.5,600,0,0
3,10,23400,PSL,NH,KLKV,24000,3.5,600,0,0
4,11,20700,PSL,NH,KLKV,21300,3.5,600,0,0
...,...,...,...,...,...,...,...,...,...,...
301,713,54000,PCD,KTNI-KDPM-MC-TVM-NTA-UDA-PLKDA-URB,KLKV,63600,59.8,9600,0,0
302,714,65700,KLKV,NH,TVM,70500,33.7,4800,0,0
303,718,60600,KLKV,NH-TVM,MC,66600,40.0,6000,0,0
304,725,38700,KLKV,KRKM,VLRD,41400,17.0,2700,0,0
